In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
import re

hn = pd.read_csv("/kaggle/input/hacker-news-posts/HN_posts_year_to_Sep_26_2016.csv")
titles = hn['title']

In [ ]:
titles

In [ ]:
hn.head()

In [ ]:
pattern = r"[Pp]ython"
python_counts = titles.str.contains(pattern).sum()
print(python_counts)

- we learned is to use re.I — the ignorecase flag — to make our pattern case insensitive:

In [ ]:
pattern = r'python'
python_counts = titles.str.contains(pattern,flags = re.I).sum()
print(python_counts)

In [ ]:
pattern = r'[Ss][Qq][Ll]'
sql_counts = titles.str.contains(pattern).sum()
print(sql_counts)

In [ ]:
pattern = r'[Ss][Qq][Ll]'
sql_counts = titles.str.contains(pattern,flags = re.I).sum()
print(sql_counts)

- Using the Series.str.extract() method.
- https://s3.amazonaws.com/dq-content/369/single_capture_group.svg

In [ ]:
pattern = r"(SQL)"
sql_capitalizations = titles.str.extract(pattern, flags=re.I)
print(sql_capitalizations)

- using the Series.value_counts() method to create a frequency table 

In [ ]:
sql_capitalizations_freq = sql_capitalizations[0].value_counts()
print(sql_capitalizations_freq)

- extend this analysis by looking at titles that have letters immediately before the "SQL," 

In [ ]:
pattern = r"(\w+SQL)"
sql_flavors = titles.str.extract(pattern,flags= re.I)
sql_flavors_freq = sql_flavors[0].value_counts()
print(sql_flavors_freq)

In [ ]:
hn_sql = hn[hn['title'].str.contains(r"\w+SQL", flags=re.I)].copy() 
hn_sql

In [ ]:
hn_sql["flavor"] = hn_sql["title"].str.extract(r"(\w+SQL)", re.I)
hn_sql["flavor"] = hn_sql["flavor"].str.lower()
sql_pivot = hn_sql.pivot_table(index="flavor",values='num_comments', aggfunc='mean')

In [ ]:
sql_pivot

- capture groups to extract the version of Python https://s3.amazonaws.com/dq-content/369/python_versions.svg
[\d\.]+ - one or more digit or character 

In [ ]:
pattern = r"[Pp]ython ([\d\.]+)"

py_versions = titles.str.extract(pattern)
py_versions_freq = dict(py_versions[0].value_counts())
py_versions_freq

- Next up: counting the mentions of the C language.
https://s3.amazonaws.com/dq-content/369/c_regex_1.svg
- \b a word boundry ..[Cc]- the character Cc \b a word boundry 
- re-useing the previous first_10_matches() function 

In [ ]:
def first_10_matches(pattern):
    """
    Return the first 10 story titles that match
    the provided regular expression
    """
    all_matches = titles[titles.str.contains(pattern)]
    first_10 = all_matches.head(10)
    return first_10

first_10_matches(r"\b[Cc]\b")

- use a negative set to prevent matches for the + character and the .

In [ ]:
def first_10_matches(pattern):
    all_matches = titles[titles.str.contains(pattern)]
    first_10 = all_matches.head(10)
    return first_10
    

pattern = r"\b[Cc]\b [^ . +]"
first_10_matches(pattern)


- Instead we'll need a new tool: lookarounds.( to avoid  irrelevant result line series C)
- https://s3.amazonaws.com/dq-content/369/lookarounds.svg - four types of lookarounds
- 1. tips
* Inside the parentheses, the first character of a lookaround is always ?.
* If the lookaround is a lookbehind, the next character will be <, which you can think of as an arrow head pointing behind the match.
* The next character indicates whether the lookaround is positive (=) or negative (!).

In [ ]:
test_cases = ['Red_Green_Blue',
              'Yellow_Green_Red',
              'Red_Green_Red',
              'Yellow_Green_Blue',
              'Green']

In [ ]:
def run_test_cases(pattern):
    for tc in test_cases:
        result = re.search(pattern, tc)
        print(result or "NO MATCH")

In [ ]:
run_test_cases(r"Green(?=_Blue)")

In [ ]:
run_test_cases(r"Green(?!_Red)")

- positive lookbehind

In [ ]:
run_test_cases(r"(?<=Red_)Green")

- negative lookbehind

In [ ]:
run_test_cases(r"(?<!Yellow_)Green")

In [ ]:
run_test_cases(r"Green(?=.{5})")

- now use lookarounds to exclude the matches we don't want

In [ ]:
first_10_matches(r"\b[Cc]\b[^.+]")

In [ ]:
pattern = r"(?<!Series\s)\b[Cc]\b(?![\+\.])"
c_mentions = titles.str.contains(pattern).sum()
c_mentions 

- using backreferences for double letters HelloGoodbye
- https://s3.amazonaws.com/dq-content/369/backreference_syntax_1.svg
- Within a regular expression, we can use a backslash followed by that integer to refer to the group:\2\1

- HelloGoodbyeGoodbyeHello - (\w)\1
- https://s3.amazonaws.com/dq-content/369/backreference_syntax_2.svg

In [ ]:
test_cases = [
              "I'm going to read a book.",
              "Green is my favorite color.",
              "My name is Aaron.",
              "No doubles here.",
              "I have a pet eel."
             ]
for tc in test_cases:
    print(re.search(r"(\w)\1", tc))

In [ ]:
test_cases = pd.Series(test_cases)
test_cases


In [ ]:
print(test_cases.str.contains(r"(\w)\1"))

In [ ]:
pattern = r"\b(\w+)\s\1\b"

repeated_words = titles[titles.str.contains(pattern)]
repeated_words

- str.replace() method is used to replace simple substrings
- We can achieve the same with regular expressions using the re.sub() function

In [ ]:
re.sub(pattern, repl, string, flags=0)

- repl parameter is the text that you would like to substitute for the match

In [ ]:
string = "aBcDEfGHIj"

print(re.sub(r"[A-Z]", "-", string))

- while working with pandas we use, Series.str.replace(pat, repl, flags=0)

- there were multiple different capitalizations for SQL in our dataset. Lets make this uniform, Series.str.replace() method and a regular expression:

In [ ]:
sql_variations = pd.Series(["SQL", "Sql", "sql"])

sql_uniform = sql_variations.str.replace(r"sql", "SQL", flags=re.I)
print(sql_uniform)

- Question here we are provided with email_variations,lets use the techique to make this uniofrm 

In [ ]:
email_variations = pd.Series(['email', 'Email', 'e Mail',
                        'e mail', 'E-mail', 'e-mail',
                        'eMail', 'E-Mail', 'EMAIL'])
pattern = r"e[\-\s]?mail"
email_uniform = email_variations.str.replace(pattern, "email", flags=re.I)
email_uniform

In [ ]:
titles_clean = titles.str.replace(pattern, "email", flags=re.I)
titles_clean

- we'll extract components of URLs from our dataset,task we will be performing first is extracting the different components of the URLs in order to analyze them. On this screen, we'll start by extracting just the domains.https://s3.amazonaws.com/dq-content/369/url_examples_1_updated.svg
* following technique:

- Using a series of characters that will match the protocol.
- Inside a capture group, using a set that will match the character classes used in the domain.
- Because all of the URLs either end with the domain, or continue with page path which starts with / (a character not found in any domains), we don't need to cater for this part of the URL in our regular expression.

In [ ]:
test_urls = pd.Series([
 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429',
 'http://www.interactivedynamicvideo.com/',
 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0',
 'http://evonomics.com/advertising-cannot-maintain-internet-heres-solution/',
 'HTTPS://github.com/keppel/pinn',
 'Http://phys.org/news/2015-09-scale-solar-youve.html',
 'https://iot.seeed.cc',
 'http://www.bfilipek.com/2016/04/custom-deleters-for-c-smart-pointers.html',
 'http://beta.crowdfireapp.com/?beta=agnipath',
 'https://www.valid.ly?param',
 'http://css-cursor.techstream.org'
])
pattern = r"https?://([\w\-\.]+)"
test_urls_clean = test_urls.str.extract(pattern,flags=re.I)
test_urls_clean

In [ ]:
domains = hn["url"].str.extract(pattern,flags= re.I)
top_domains = domains[0].value_counts().head(5)
top_domains

[](http://)Having extracted just the domains from the URLs, on this final screen we'll extract each of the three component parts of the URLs:

* Protocol
* Domain
* Page path

https://s3.amazonaws.com/dq-content/369/url_examples_2_updated.svg

In [ ]:
created_at = hn['created_at'].head()
print(created_at)

- We'll use capture groups to extract these dates and times into two columns:
- https://s3.amazonaws.com/dq-content/369/multiple_capture_groups.svg
- (.+)\s(.+) 

In [ ]:
pattern = r"(.+)\s(.+)"
dates_times = created_at.str.extract(pattern)
print(dates_times)

- Question write a regular expression that will extract the URL components into individual columns of a dataframe
- Write a regular expression that extracts URL components using three capture groups:
* The first capture group should include the protocol text, up to but not including ://.
* The second group should contain the domain, from after :// up to but not including /.
* The third group should contain the page path, from after / to the end of the string.

In [ ]:
pattern = r"(https?)://([\w\.\-]+)/?(.*)"

test_url_parts = test_urls.str.extract(pattern, flags=re.I)
test_url_parts

In [ ]:
url_parts = hn['url'].str.extract(pattern, flags=re.I)
url_parts

- final task will be to name unnamed columns using named capture groups

In [ ]:
#e.g
created_at = hn['created_at'].head()

pattern = r"(.+) (.+)"
dates_times = created_at.str.extract(pattern)
print(dates_times)

- https://s3.amazonaws.com/dq-content/369/named_capture_groups.svg - In order to name a capture group we use the syntax ?P, where name is the name of our capture group

In [ ]:
pattern = r"(?P<date>.+) (?P<time>.+)"
dates_times = created_at.str.extract(pattern)
print(dates_times)

- Question adding names to our capture group from the previous screen to create a dataframe with named columns.

In [ ]:
pattern = r"(?P<protocol>https?)://(?P<domain>[\w\.\-]+)/?(?P<path>.*)"
url_parts = hn['url'].str.extract(pattern,flags=re.I)
url_parts